## TRABAJO FINAL:  Sistema de Recomendación de peliculas usando VSM y Similitud Coseno
### Autor:  Wilmar Martín

In [10]:
# Autor: Wilmar Martín

import math
import json
from time import time
from collections import Counter

In [11]:
def opendDocuments():    
    file_inverdIndex = open("AnalisisGenerados/inverdIndexGen.txt", "r")
    inverdIndex = json.JSONDecoder().decode(file_inverdIndex.read())
    file_inverdIndex.close()
    
    file_stopwords = open("AnalisisGenerados/stopwordsGen.txt", "r", errors="replace")
    stopwords = file_stopwords.read().split()
    file_stopwords.close()
    
    file_words = open("AnalisisGenerados/wordsGen.txt", "r")
    words = json.JSONDecoder().decode(file_words.read())
    file_words.close()
    
    file_words = open("AnalisisGenerados/tfidf.txt", "r")
    allTfidf = json.JSONDecoder().decode(file_words.read())
    file_words.close()

    file_documents = open("documents/documentsGen.txt", "r")
    documents = json.JSONDecoder().decode(file_documents.read())
    file_documents.close()
        
    file_inv_frec_vector = open("AnalisisGenerados/inv_frec_vector.txt", "r")
    inv_frec_vector = json.JSONDecoder().decode(file_inv_frec_vector.read())
    file_inv_frec_vector.close()
    
    return inverdIndex, stopwords, words, allTfidf ,documents, inv_frec_vector

In [12]:
def findCoincidences(doc, find_term):
    coincidences = 0
    for word in doc:
        if word == find_term:
            coincidences = coincidences + 1
    return coincidences

In [13]:
def cosine_similarity(vectorSpace1, vectorSpace2):
    numerator = 0 
    sumxx, sumyy = 0, 0
    for i in range(len(vectorSpace1)):
        x = vectorSpace1[i]
        y = vectorSpace2[i]
        sumxx += x*x
        sumyy += y*y
        numerator += x*y
    return numerator/math.sqrt(sumxx*sumyy)

In [14]:
def createHistogram(query):
    listQuery = []
    for word in query.lower().split():  # split
        if word not in stopwords:
            listQuery.append(word)
    return Counter(listQuery)


In [15]:
def createVectorSpace(histogram):
    vectorSpace = []
    for word in words:
        vectorSpace.append(findCoincidences(histogram, word))
    return vectorSpace

In [16]:
def createTdiDf(vectorSpace):
    tfidf = []
    for id, ter_frec in enumerate(vectorSpace):
        eq = 0
        if ter_frec > 0:
            eq = ter_frec*inv_frec_vector[id]
        tfidf.append(eq)
    return tfidf

In [17]:
def search(tfidf):
    cosSim = {}
    for palabra in histQuery:
        if palabra in inverdIndex:  # si la palabra esta en el index invertido
            for key in inverdIndex.get(palabra):
                if key not in cosSim:
                    calc = cosine_similarity(tfidf, allTfidf[key])
                    cosSim[key] = calc                    
    return cosSim

### Algoritmo de Búsqueda

In [18]:
# abrir documentos con los documentos procesados
inverdIndex, stopwords, words, allTfidf ,documents, inv_frec_vector = opendDocuments()

query = input('Ingrese texto a buscar: ')

# obtiene el tiempo actual
t0 = time()

# crea al vector de histograma de la cosulta
histQuery = createHistogram(query)

# crea el vector space de la consulta
vectorSpace = createVectorSpace(histQuery)

# normaliza TF-IDF 
tfidf = createTdiDf(vectorSpace)

# realiza la consulta
docs = search(tfidf)

# obtiene el tiempo total de la busqueda
totalTime = time()-t0

# muestra resultados
print()
print("Tiempo total de la busqueda %0.3fs." % totalTime)
print("Total de Peliculas encontradas: %d" % len(docs))
print("La consulta es: " + query)
print()

i = 0
for key in sorted(docs, key=docs.get, reverse=True):
    print("Pelicula encontrada: #%s, cs: %f" % (i, docs[key]))
    print('Pelicula #%s:' % (key))
    str2 = "sinopsis:"
    print('%s' % ( documents[key][0:documents[key].rfind(str2)].replace("titulo", "TITULO")))
    print('%s' % ( documents[key][documents[key].rfind(str2):len(documents[key])].replace("sinopsis", "SINOPSIS")))
    print()
    i += 1
    
    if (i > 10):
        break

Ingrese texto a buscar: guerra

Tiempo total de la busqueda 0.179s.
Total de Peliculas encontradas: 64
La consulta es: guerra

Pelicula encontrada: #0, cs: 0.168785
Pelicula #256:
TITULO: en honor a la verdad 
SINOPSIS: guerra de irak. después de destruir accidentalmente uno de sus propios tanques, el teniente coronel nathan serling (denzel washington) vuelve a casa. el ejército oculta el incidente y asigna a serling la misión de investigar al capitán karen walden (meg ryan), una piloto de helicóptero que, a título póstumo, está a punto de convertirse en la primera mujer héroe de guerra que recibe la medalla al honor por una acción en combate. ésta fue la primera película que se hizo en hollywood sobre la i guerra del golfo. 

Pelicula encontrada: #1, cs: 0.167080
Pelicula #251:
TITULO: gracias por su servicio 
SINOPSIS: este drama cuenta la historia de unos veteranos de guerra que, al retornar del campo de batalla, procurarán integrarse nuevamente en la sociedad. mas amoldarse a la vi